# 网络与Web编程

## 作为客户端与HTTP服务交互

你需要通过HTTP协议以客户端的方式访问多种服务。例如，下载数据或者与基于REST的API进行交互

对于简单的事情来说，通常使用urllib.request模块就够了。例如，发送一个简单的HTTPGET请求到远程的服务上，可以这样做

In [3]:
from urllib import request, parse

url = 'http://httpbin.org/get'

parms = {
    'name1': 'value1',
    'name2': 'value2'
}

querystring = parse.urlencode(parms)

u = request.urlopen(url + '?' + querystring)
resp = u.read()

In [4]:
resp

b'{\n  "args": {\n    "name1": "value1", \n    "name2": "value2"\n  }, \n  "headers": {\n    "Accept-Encoding": "identity", \n    "Host": "httpbin.org", \n    "User-Agent": "Python-urllib/3.7"\n  }, \n  "origin": "114.242.9.141, 114.242.9.141", \n  "url": "https://httpbin.org/get?name1=value1&name2=value2"\n}\n'

如果你需要使用POST方法在请求主体中发送查询参数，可以将参数编码后作为可选参数提供给urlopen()函数，就像这样：

In [7]:
from urllib import request, parse

url = 'http://httpbin.org/post'
parms = {
    'name1': 'value1',
    'name2': 'value2'
}
querystring = parse.urlencode(parms, querystring.encode('ascii'))
resp = u.read()


In [8]:
resp

b''

如果你需要在发出的请求中提供一些自定义的HTTP头，例如修改user-agent字 段,可以创建一个包含字段值的字典，并创建一个Request实例然后将其传给urlopen()，如下：

In [10]:
from urllib import request, parse

headers = {
    'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:69.0) Gecko/20100101 Firefox/69.0',
    'Spam': 'Eggs'
}

req = request.Request(url, querystring.encode('ascii'), headers=headers)

u = request.urlopen(req)
resp = u.read()

如果需要交互的服务比上面的例子都要复杂，也许应该去看看requests库（https://pypi.python.org/pypi/requests）。例如，下面这个示例采用requests库重新实现了上面的操作：

In [11]:
import requests

url = 'http://httpbin.org/post'
parms = {
    'name1': 'value1',
    'name2': 'value2'
}

headers = {
    'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:69.0) Gecko/20100101 Firefox/69.0',
    'Spam': 'Eggs'
}

resp = requests.post(url, data=parms, headers=headers)
text = resp.text

关于requests库，一个值得一提的特性就是它能以多种方式从请求中返回响应结果的内容。从上面的代码来看，resp.text带给我们的是以Unicode解码的响应文本。但是，如果去访问resp.content，就会得到原始的二进制数据。另一方面，如果访问 
resp.json，那么就会得到JSON格式的响应内容。

下面这个示例利用requests库发起一个HEAD请求，并从响应中提取出一些HTTP头数据的字段：

In [12]:
text

'{\n  "args": {}, \n  "data": "", \n  "files": {}, \n  "form": {\n    "name1": "value1", \n    "name2": "value2"\n  }, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Content-Length": "25", \n    "Content-Type": "application/x-www-form-urlencoded", \n    "Host": "httpbin.org", \n    "Spam": "Eggs", \n    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:69.0) Gecko/20100101 Firefox/69.0"\n  }, \n  "json": null, \n  "origin": "114.242.9.141, 114.242.9.141", \n  "url": "https://httpbin.org/post"\n}\n'

In [13]:
resp.content

b'{\n  "args": {}, \n  "data": "", \n  "files": {}, \n  "form": {\n    "name1": "value1", \n    "name2": "value2"\n  }, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Content-Length": "25", \n    "Content-Type": "application/x-www-form-urlencoded", \n    "Host": "httpbin.org", \n    "Spam": "Eggs", \n    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:69.0) Gecko/20100101 Firefox/69.0"\n  }, \n  "json": null, \n  "origin": "114.242.9.141, 114.242.9.141", \n  "url": "https://httpbin.org/post"\n}\n'

In [15]:
(resp.json)['args']

TypeError: 'method' object is not subscriptable

In [16]:
resp.json

<bound method Response.json of <Response [200]>>

下面这个示例利用requests库发起一个HEAD请求，并从响应中提取出一些HTTP头数据的字段：

In [20]:
import requests

resp = requests.head('http://www.python.org/index.html')

status = resp.status_code  # 301
last_modified = resp.headers['Last-Modified']  # Not Have
content_type = resp.headers['content-type']  # Not Have
content_length = resp.headers['content-length']  # Not Have
status, last_modified, content_type, content_length

KeyError: 'last-modified'

In [22]:
resp.headers

{'Server': 'Varnish', 'Retry-After': '0', 'Location': 'https://www.python.org/index.html', 'Content-Length': '0', 'Accept-Ranges': 'bytes', 'Date': 'Thu, 17 Oct 2019 09:33:20 GMT', 'Via': '1.1 varnish', 'Connection': 'close', 'X-Served-By': 'cache-hnd18739-HND', 'X-Cache': 'HIT', 'X-Cache-Hits': '0', 'X-Timer': 'S1571304800.227863,VS0,VE0', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains'}

.......

坦白说，所有的这些操作在requests库中都变得简单的多

在开发过程中测试HTTP客户端代码常常是很令人沮丧的，因为所有棘手的细节 问题都需要考虑（例如cookies、认证、HTTP头、编码方式等）。要完成这些任务，考虑使用httpbin服务（http://httpbin.org）。这个站点会接收发出的请求，然后以JSON 的形式将相应信息回传回来。下面是一个交互式的例子：

In [28]:
import requests

r = requests.get(
    'http://httpbin.org/get?name=Dave&n=37',
    headers = {'User-agent': 'goaway/1.0'}
)

resp = r.json() # 原文报错 r.json
resp['headers']

{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate',
 'Host': 'httpbin.org',
 'User-Agent': 'goaway/1.0'}

In [24]:
resp

<bound method Response.json of <Response [200]>>

In [25]:
r

<Response [200]>

In [26]:
r.json

<bound method Response.json of <Response [200]>>

In [29]:
resp['headers']

{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate',
 'Host': 'httpbin.org',
 'User-Agent': 'goaway/1.0'}

In [30]:
resp['args']

{'n': '37', 'name': 'Dave'}

在要同一个真正的站点进行交互前，先在httpbin.org这样的网站上做实验常常是可取的办法。尤其是当我们面对3次登录失败就会关闭账户这样的风险时尤为有用（不要尝试自己编写HTTP认证客户端来登录你的银行账户）。

尽管本节没有涉及，request库还对许多高级的HTTP客户端协议提供了支持，比如OAuth。requests模块的文档（http://docs.python-requests.org)质量很高（坦白说比在这短短的一节的篇幅中所提供的任何信息都好），可以参考文档以获得更多地信息。

## 创建TCP服务器

你想实现一个服务器，通过TCP协议和客户端通信

创建一个TCP服务器的一个简单方法是使用socketserver库。例如，下面是一个简单的应答服务器

In [ ]:
from socketserver import BaseRequestHandler, TCPServer

class EchoHandler(BaseRequestHandler):
    def handle(self):
        print('Got connection from', self.client_address)
        while True:
            msg = self.request.recv(8192)
            if not msg:
                break
            self.request.send(msg)
            
if __name__ == '__main__':
    serv = TCPServer(('', 20000), EchoHandler)
    serv.serve_forever()

Got connection from ('127.0.0.1', 55603)


在这段代码中，你定义了一个特殊的处理类，实现了一个handle()方法，用来为客户端连接服务。request属性是客户端socket，client_address有客户端地址。为了测试这个服务器，运行它并打开另外一个Python进程连接这个服务器：

In [ ]:
from socket import socket, AF_INET, SOCK_STREAM

In [ ]:
s = socket(AF_INET, SOCK_STREAM)
s.connect(('localhost', 20000))
s.send(b'hello')  #  5

In [ ]:
s.recv(8192)  # b'hello'